# Semantic Segmentation 과제
> 인공지능 스터디 다섯 번째 과제에 오신 것을 환영합니다! 강의를 들으면서 배운 다양한 지식들을 실습을 통해서 활용해 볼 시간을 가질 것입니다!

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> Semantic Segmentation 과 Instance Segmentation (주관식)
```python
Semantic Segmentation 과 Instance Segmentation의 차이에 대해 간단히 적어주세요.
```

```python
😉
# TODO :
Semantic Segmentation은 이미지 내의 각 픽셀을 레이블 된 클래스로 분류하는 작업이고 Instance Segmentation은 Semantic Segmentation과 객체 간에 구분을 하면서 더욱 정확한 객체의 경계를 식별하는 작업이다.

```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> FCN (주관식)
```python
FCN 모델에서 image classification 모델과 다르게 마지막 레이어까지 위치정보를 유지한 방법을 간단히 적어주세요.
```

```python
😉
# TODO :
1*1 conv layer, 업샘플링 레이어, Skip Connections
```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> DeconvNet (주관식)
```python
FCN의 한계점을 보완하기 위해 DeconvNet이 사용했던 방법을 간단히 적어주세요.
```

```python
😉
# TODO :
업샘플링 레이어 대신 Deconvolution 레이어 사용.
-> 인코더와 디코더 구조를 사용해 Unpooling과 Transpose Convolution block을 반복

```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> Transposed Convolution (주관식)
```python
다음 Transpose Convolution 연산의 결과로 나온 output 행렬에서 (1), (2), (3), (4)에 들어갈 숫자를 적어주세요.
```
![](https://github.com/Pjunn/GDSC_mlstudy/blob/main/5%EC%A3%BC%EC%B0%A8/TransposeConv.png?raw=true)


```python
😉
# TODO : 1
# TODO : 0
# TODO : 0
# TODO : 1

```

#### ❓ <font color='red'><b>[ 퀴즈 ]</b></font> Unpooling (주관식)
```python
다음 Maxpooling 과 Max Unpooling 연산의 결과로 나온 output 행렬에서 (1), (2), (3), (4)에 들어갈 숫자를 적어주세요.

```
![](https://github.com/Pjunn/GDSC_mlstudy/blob/main/5%EC%A3%BC%EC%B0%A8/Unpooling.png?raw=true)

```python
😉
# TODO : 7
# TODO : 8
# TODO : 7
# TODO : 0

```

```python
🐙
vgg16의 구조를 그대로 사용하여 image classification 모델에서 semantic segmentation을 해결하기 위한 모델로 변경을 시도한 FCN을 구현해 보아요.
그중에서 FCN 32s 모델을 구현해 볼 거예요.
```

### 👨‍💻 <font color='green'><b>[ 코딩 ]</b></font> FCN 32s 구현

![FCN32-VGG16.png](http://drive.google.com/uc?export=view&id=1mGO5xoQdkpkt0ntz-hydr62MriK8YCLE)

```python
위 이미지는 FCN32의 전체적인 구조입니다. Classification을 위하여 사용하였던 마지막 fully connected layer는 convolution backbone을 통해 얻은 feature map을 펼친 다음 (flatten) 연산을 진행하기 때문에 입력 이미지의 공간 정보(spatial information)를 유지하지 못합니다. 따라서 FCN에서는 fully connected layer를 1x1 convolution으로 바꾸면서 공간 정보를 유지하며 segmentation을 수행합니다.
```

```python
먼저 주어진 FCN32s 클래스를 살펴봅시다.

__init__ 에서는 num_classes를 인자로 받게 되는데 이는 본 세그멘테이션 모델이 구별해야 하는 클래스 수입니다.

CR 함수는 위 이미지의 Convolution block을 구성하는 노란색 박스를 정의한 것입니다. 이 함수는 순서대로 모듈을 실행하는 데에 nn.Sequential을 사용합니다. 처음에는 Conv 모듈을 실행하고, 그 다음에는 ReLU 모듈을 실행합니다.

FCN32s에서 conv5 block 까지는 VGG16의 구조와 동일합니다.
VGG16에 대해 알아봅시다.
```


![](https://wikidocs.net/images/page/164796/Fig_05.png)

```python
VGG16의 구조도 입니다. VGG16은 5개의 블록 / 각 블록은 2, 2, 3, 3, 3 개의 컨볼루션 레이어 / 각 블록의 컨볼루션 레이어는 64, 128, 256, 512, 512 개의 필터 수를 가지고 있습니다.
이때의 stride, padding은 모두 1로 설정되어 있어 이미지의 사이즈가 변하지 않습니다.
각 블록의 마지막단에는 maxpooling이 적용되었습니다.
2x2 maxpooling이 적용되면서 5번의 블록을 거칠 때 이미지의 사이즈가 (1/2) ** 5 = 1/32 사이즈로 줄어들게 됩니다.

FCN32s에서 conv1 block에 해당하는 코드를 볼 수 있습니다.
이를 참고하여 conv5 block까지 완성해 봅시다.

FCN32s에서 fc6 과 fc7 레이어는 은 1x1 convolution을 사용합니다. 각 레이어는 4096 개의 필터 수를 가지고 있습니다.
이때 VGG16의 Fully connected layer 1, 2 와 파라미터 수가 같기 때문에 VGG16의 파라미터를 reshape 하여 그대로 사용합니다.
score 레이어 또한 1x1 convolution을 사용하고 필터 수는 클래스의 개수를 사용하며 아웃풋의 각 셀은 각 클래스에 대한 score를 가지고 있습니다.

fc6 와 fc7 또한 CR을 사용하여 완성해 봅시다.
하지만 score 레이어에서는 score 결과를 가지기 때문에 ReLU 를 거치지 않으므로 nn.Conv2d 만을 사용하여 구현해 주세요.

이후에는 Transpose Convolution 을 사용하여 업 샘플링을 진행합니다.
이때 FCN32s의 32s는 여기 Transposed Convolution의 Stride 32를 의미합니다.
```

In [1]:
# torch
import torch
import torch.nn as nn

In [8]:
class FCN32s(nn.Module):
    def __init__(self, num_classes):
        super(FCN32s, self).__init__()

        def CR(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
            return nn.Sequential(nn.Conv2d(in_channels=in_channels,
                                            out_channels=out_channels,
                                            kernel_size=kernel_size,
                                            stride=stride,
                                            padding=padding),
                                  nn.ReLU(inplace=True)
                                 )

        # conv1
        self.conv1_1 = CR(3, 64, 3, 1, 1)
        self.conv1_2 = CR(64, 64, 3, 1, 1)
        self.pool1 = nn.MaxPool2d(2, stride=2, ceil_mode=True)


        # [TODO]
        # conv2
        self.conv2_1 = CR(64, 128, 3, 1, 1)
        self.conv2_2 = CR(128, 128, 3, 1, 1)
        self.pool2 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        # conv3
        self.conv3_1 = CR(128, 256, 3, 1, 1)
        self.conv3_2 = CR(256, 256, 3, 1, 1)
        self.conv3_3 = CR(256, 256, 3, 1, 1)
        self.pool3 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        # conv4
        self.conv4_1 = CR(256, 512, 3, 1, 1)
        self.conv4_2 = CR(512, 512, 3, 1, 1)
        self.conv4_3 = CR(512, 512, 3, 1, 1)
        self.pool4 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        # conv5
        self.conv5_1 = CR(512, 512, 3, 1, 1)
        self.conv5_2 = CR(512, 512, 3, 1, 1)
        self.conv5_3 = CR(512, 512, 3, 1, 1)
        self.pool5 = nn.MaxPool2d(2, stride=2, ceil_mode=True)

        # FC6, FC7 (1x1 convolution)
        self.fc6 = CR(512, 4096, 1, 1, 0)
        self.fc7 = CR(4096, 4096, 1, 1, 0)

        # Score Layer (1x1 convolution)
        self.score = nn.Conv2d(4096, num_classes, 1, 1, 0)

        # Transpose Conv를 사용한 업샘플링
        self.upscore32 = nn.ConvTranspose2d(4096, num_classes, kernel_size=64, stride=32, padding=16)

    def forward(self, x):
        #[TODO]
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.pool1(x)

        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.pool2(x)

        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.pool3(x)

        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.pool4(x)

        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)
        x = self.pool5(x)

        x = self.fc6(x)
        x = self.fc7(x)
        score = self.score(x)

        output = self.upscore32(x)
        return output

```python
🐙
구현한 FCN32s 모델의 output shape을 확인해 보아요.

모델의 아웃풋은 각 channel이 하나의 class에 대한 값을 가지고 입력 이미지의 크기와 동일한 사이즈 (batch_size, num_classes, height, width)의 shape을 가져요.

test_input은 batch_size가 1이고 input resolution이 224 x 224 인 Tensor입니다.
아래의 코드를 실행하여 segmentation model의 출력 크기가 정확한지 확인해 보아요.
```



In [9]:
test_input = torch.randn((1, 3, 224, 224))

model = FCN32s(num_classes = 5) # class 개수를 5라고 가정해봅시다
output = model(test_input)
print('The output shape of the segmentation network:', output.shape)

# 아래 코드는 수정하실 필요가 없습니다!
if output.shape == (1, 5, 224, 224):
    print("🎉🎉🎉 성공!!! 🎉🎉🎉")
else:
    print("🐙 다시 도전해봐요!")

The output shape of the segmentation network: torch.Size([1, 5, 224, 224])
🎉🎉🎉 성공!!! 🎉🎉🎉


```python
🐙
구현하느라 고생 많았어요!
우리가 구현한 방식과 비슷하게 FCN16s 모델과 FCN8s 모델 또한 구현해 볼 수 있어요.
두 모델의 구조는 다음 이미지에서 확인할 수 있어요.
```

### FCN16s
![FCN16s.png](http://drive.google.com/uc?export=view&id=1Td6ZLqY3A5Mwx9vwLJAyJytNW1ILhZjn)

### FCN8s
![FCN8s.png](http://drive.google.com/uc?export=view&id=1MqoHuPys35fvTEu1Cv0KQ6Y3eoYGiQb0)

```python
🐙
여기서 우리가 구현한 FCN32s를 생각해 봤을 때 입력 이미지를 32배 만큼 줄이고 거기서 바로 32배 만큼 키우는 것이 괜찮을까요?
너무 한 번에 사이즈를 키우는 것이 아닌가라는 의문이 들 수 있어요.
여기서 FCN 논문에서는 skip connection을 적용한 FCN16s, FCN8s를 제시하였고 성능 향상을 확인할 수 있었어요.
skip connection을 통해서 얻을 수 있는 이점은 2가지예요
1. MaxPooling에 의해서 잃어버린 정보를 Summation 연산을 통해 복원해 주는 작업을 진행하였다.
2. 32배 -> 2배, 16배 와 같이 Upsampled Size를 줄여주기에 좀 더 효율적인 이미지 복원이 가능하였다.
```

![comparison.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F0OEue%2FbtqSTJY8AH6%2FhVkgLTafKAb4hPlqZMrzi0%2Fimg.png)

```python
실제 결과를 확인해 보았을 때 FCN32s에서 FCN8s로 갈수록 더 디테일한 Segmentation 이미지를 얻는 것을 확인할 수 있어요.
```

### 🎉🎉🎉 5주차 과제 완료! 🎉🎉🎉
```python
🐙
여러분 모두 수고 했어요!!
```